In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
df_cust = spark.read.table('data_warehouse.bronze.cust_info')

In [0]:
window_spec = Window.partitionBy("cst_id").orderBy(col("cst_create_date").desc())

df_cust = (
    df_cust.withColumn("rn", row_number().over(window_spec))
      .filter(col("rn") == 1)
      .drop("rn")
)


In [0]:
df_cust = df_cust.withColumn("cst_firstname", trim(col("cst_firstname"))) \
             .withColumn("cst_lastname", trim(col("cst_lastname")))


In [0]:
df_cust = df_cust.withColumn(
    "cst_marital_status",
    when(upper(trim(col("cst_marital_status"))) == "S", "Single")
     .when(upper(trim(col("cst_marital_status"))) == "M", "Married")
     .otherwise("Unknown")
)


In [0]:
df_cust = df_cust.withColumn(
    "cst_gndr",
    when(upper(trim(col("cst_gndr"))) == "M", "Male")
     .when(upper(trim(col("cst_gndr"))) == "F", "Female")
     .otherwise("Unknown")
)

In [0]:
df_cust.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("data_warehouse.silver.cust_info")


In [0]:
%sql
SELECT * FROM data_warehouse.silver.cust_info